#Processes and Threads

#Processes

In [ ]:
import multiprocessing

def square(numbers, result):
    for idx, n in enumerate(numbers):
        result[idx] = n * n

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5]
    result = multiprocessing.Array('i', len(numbers))

    process = multiprocessing.Process(target=square, args=(numbers, result))
    process.start()
    process.join()

    print("Squared numbers:", list(result))

Squared numbers: [1, 4, 9, 16, 25]


#Threads

In [ ]:
import threading

def greet():
    print("Hello from a thread!")

if __name__ == "__main__":
    thread = threading.Thread(target=greet)
    thread.start()
    thread.join()

Hello from a thread!


#Concurrency: Mutual Exclusion and Synchronization Mechanisms

Concurrency involves managing multiple tasks that execute independently but may interact with shared resources. Key mechanisms to ensure correct behavior include mutual exclusion and synchronization.



#Locks and Semaphores

In [ ]:
import threading

shared_resource = 0
lock = threading.Lock()

def modify_shared_resource():
    global shared_resource
    with lock:
        for _ in range(10000):
            shared_resource += 1

# Create multiple threads to modify the shared resource
threads = []
for _ in range(5):
    thread = threading.Thread(target=modify_shared_resource)
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

print("Final shared resource value:", shared_resource)

Final shared resource value: 50000


In [ ]:
import threading
import time

semaphore = threading.Semaphore(2)  # Allow 2 threads to access the resource simultaneously
resource = []

def produce():
    global resource
    while True:
        with semaphore:
            resource.append('item')
            print("Produced 1 item")
            time.sleep(1)

def consume():
    global resource
    while True:
        with semaphore:
            if resource:
                resource.pop()
                print("Consumed 1 item")
            time.sleep(1)

# Create producer and consumer threads
producer_thread = threading.Thread(target=produce)
consumer_thread = threading.Thread(target=consume)

producer_thread.start()
consumer_thread.start()

time.sleep(10)  # Run for 10 seconds

Produced 1 item
Consumed 1 item
Produced 1 item
Consumed 1 item
Produced 1 item
Consumed 1 item
Produced 1 item
Consumed 1 item
Produced 1 item
Consumed 1 item
Produced 1 item
Consumed 1 item
Produced 1 item
Consumed 1 item
Produced 1 item
Consumed 1 item
Produced 1 item
Consumed 1 item
Produced 1 item
Consumed 1 item
Produced 1 item
Consumed 1 item


#Concurrency and Mutual Exclusion/Synchronization Mechanisms

#Producer and Consumer

In [ ]:
import queue
import threading
import time
import random

# Shared queue with a maximum size
shared_queue = queue.Queue(maxsize=10)

# Sentinel value to indicate the end of production
SENTINEL = object()

# Function representing the producer
def producer():
    for i in range(20):  # Produce 20 items
        item = random.randint(1, 100)
        shared_queue.put(item)
        print(f"Produced {item}")
        time.sleep(random.uniform(0.5, 1.0))  # Simulate some delay
    # Put the sentinel value to indicate the end
    shared_queue.put(SENTINEL)

# Function representing the consumer
def consumer():
    while True:
        try:
            item = shared_queue.get(timeout=2)  # Timeout to allow termination
            if item is SENTINEL:
                # Reached the end
                shared_queue.task_done()
                break
            print(f"Consumed {item}")
            shared_queue.task_done()
            time.sleep(random.uniform(0.5, 1.0))  # Simulate some processing time
        except queue.Empty:
            # If queue is empty for too long, assume end of production
            break

# Create producer and consumer threads
producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)

# Start the threads
producer_thread.start()
consumer_thread.start()

# Wait for the producer to finish
producer_thread.join()

# Wait for the consumer to finish (with a timeout)
consumer_thread.join(timeout=3)

print("Production and consumption complete.")

Reader Thread-27 (reader_task) is reading resource: 4
Produced 30
Consumed 30
Consumed 1 item
Produced 1 item
Reader Thread-26 (reader_task) is reading resource: 4
Produced 23
Consumed 23
Produced 80
Consumed 1 item
Produced 1 item
Consumed 80
Reader Thread-27 (reader_task) is reading resource: 4
Produced 13
Consumed 1 item
Produced 1 item
Consumed 13
Produced 53
Reader Thread-26 (reader_task) is reading resource: 4
Consumed 53
Consumed 1 item
Produced 1 item
Produced 70
Reader Thread-27 (reader_task) is reading resource: 4
Consumed 70
Consumed 1 item
Produced 1 item
Produced 67
Reader Thread-26 (reader_task) is reading resource: 4
Consumed 67
Produced 10
Consumed 1 item
Produced 1 item
Consumed 10
Reader Thread-27 (reader_task) is reading resource: 4
Produced 29
Produced 19
Consumed 1 item
Produced 1 item
Consumed 29
Reader Thread-26 (reader_task) is reading resource: 4
Consumed 19
Produced 79
Consumed 1 item
Produced 1 item
Reader Thread-27 (reader_task) is reading resource: 4
Consum

#Monitor Implementation

In [ ]:
import threading

class SharedResourceMonitor:
    def __init__(self):
        self.resource = None
        self.condition = threading.Condition()

    def set_resource(self, value):
        with self.condition:
            while self.resource is not None:
                self.condition.wait()
            self.resource = value
            self.condition.notify_all()

    def get_resource(self):
        with self.condition:
            while self.resource is None:
                self.condition.wait()
            value = self.resource
            self.resource = None
            self.condition.notify_all()
            return value

def producer(monitor):
    for i in range(5):
        monitor.set_resource(i)
        print(f"Produced: {i}")

def consumer(monitor):
    for _ in range(5):
        value = monitor.get_resource()
        print(f"Consumed: {value}")

# Create shared resource monitor
monitor = SharedResourceMonitor()

# Create producer and consumer threads
producer_thread = threading.Thread(target=producer, args=(monitor,))
consumer_thread = threading.Thread(target=consumer, args=(monitor,))

producer_thread.start()
consumer_thread.start()

producer_thread.join()
consumer_thread.join()

Produced: 0
Consumed: 0Produced: 1

Consumed: 1Produced: 2

Consumed: 2Produced: 3

Consumed: 3Produced: 4

Consumed: 4


# The Reader-Writer Problem

In [ ]:
import threading
import time

class ReaderWriter:
    def __init__(self):
        self.lock = threading.Lock()
        self.readers = 0
        self.resource = 0
        self.resource_lock = threading.Lock()  # Lock for resource access

    def read(self):
        with self.lock:
            self.readers += 1
            if self.readers == 1:
                self.resource_lock.acquire()  # First reader acquires resource lock
        # Release the main lock to allow other readers to increment 'readers'
        print(f"Reader {threading.current_thread().name} is reading resource: {self.resource}")
        time.sleep(1)
        with self.lock:
            self.readers -= 1
            if self.readers == 0:
                self.resource_lock.release()  # Last reader releases resource lock

    def write(self, value):
        self.resource_lock.acquire()  # Acquire resource lock for exclusive write
        self.resource = value
        print(f"Writer {threading.current_thread().name} is writing resource: {self.resource}")
        self.resource_lock.release()  # Release resource lock after writing

# Create reader-writer instance
rw = ReaderWriter()

# Function for reader task
def reader_task(iterations):
    for _ in range(iterations):
        rw.read()
        time.sleep(0.5)

# Function for writer task
def writer_task(iterations):
    for i in range(iterations):
        rw.write(i)
        time.sleep(2)

# Define the number of iterations for each task
num_reader_iterations = 5
num_writer_iterations = 5

# Create reader and writer threads
readers = [threading.Thread(target=reader_task, args=(num_reader_iterations,)) for _ in range(2)]
writers = [threading.Thread(target=writer_task, args=(num_writer_iterations,)) for _ in range(2)]

# Start reader and writer threads
for reader in readers:
    reader.start()

for writer in writers:
    writer.start()

# Join all reader and writer threads
for reader in readers:
    reader.join()

for writer in writers:
    writer.join()

print("All tasks completed.")

Reader Thread-32 (reader_task) is reading resource: 0
Reader Thread-33 (reader_task) is reading resource: 0
Reader Thread-26 (reader_task) is reading resource: 4
Reader Thread-27 (reader_task) is reading resource: 0
Produced 1 item
Consumed 1 item
Reader Thread-32 (reader_task) is reading resource: 0
Reader Thread-33 (reader_task) is reading resource: 0
Reader Thread-26 (reader_task) is reading resource: 0
Produced 1 item
Consumed 1 item
Reader Thread-27 (reader_task) is reading resource: 0
Produced 1 item
Consumed 1 item
Reader Thread-32 (reader_task) is reading resource: 0
Reader Thread-33 (reader_task) is reading resource: 0
Reader Thread-26 (reader_task) is reading resource: 0
Reader Thread-27 (reader_task) is reading resource: 0
Consumed 1 item
Produced 1 item
Reader Thread-32 (reader_task) is reading resource: 0
Reader Thread-33 (reader_task) is reading resource: 0
Reader Thread-26 (reader_task) is reading resource: 0
Produced 1 item
Consumed 1 item
Reader Thread-27 (reader_task)